## Preparation

First, we'll define a function that we will use when building our agent.

It will generate fake weather data:

In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

## Q1. Define function description

In [ ]:
# get_weather_tool = {
#     "type": "function",
#     "name": "<TODO1>",
#     "description": "<TODO2>",
#     "parameters": {
#         "type": "object",
#         "properties": {
#             "<TODO3>": {
#                 "type": "string",
#                 "description": "<TODO4>"
#             }
#         },
#         "required": [TODO5],
#         "additionalProperties": False
#     }
# }

get_weather_tool = {
    "type": "function",
    "name": "get_weather",  # TODO1: a concise, unique function name
    "description": "Get the current weather for a specified city",  # TODO2: human-readable summary
    "parameters": {
        "type": "object",
        "properties": {
            "city": {                           # TODO3: Name of property
                "type": "string",
                "description": "The name of the city to get weather for"  # TODO4: explain the parameter
            }
        },
        "required": ["city"],  # TODO5: list of required params
        "additionalProperties": False
    }
}


TODO1: get_weather   (a concise, unique function name)

TODO2: Get the current weather for a specified city   (a nice description)

TODO3: city   (a property/attribute relevant for the function)

TODO4: The name of the city to get weather for   (Property/Attribute description)

TODO5: ["city"]   (List of required params)


## Testing it (Optional)
If you have OpenAI API Key (or alternative provider), let's test it.
A question could be "What's the weather like in Germany?"
Experiment with different system prompts to have better answers from the system.

## Q2. Adding another tool

Let's add another tool - a function that can add weather data to our database:

In [ ]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [5]:
## Description for this tool.
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set or update the weather temperature for a given city in the database",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to set the weather for"
            },
            "temp": {
                "type": "number",
                "description": "The temperature value in Celsius"
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}


Now let's write a description for it.

Optionally, you can test it after adding this function.

## MCP
MCP stands for Model-Context Protocol. It allows LLMs communicate with different tools (like Qdrant). It's function calling, but one step further:

A tool can export a list of functions it has
When we include the tool to our Agent, we just need to include the link to the MCP server

## Q3. Install FastMCP

Let's install a library for MCP - FastMCP

What's the version of FastMCP you installed?

In [4]:
# !pip install fastmcp

In [3]:
!pip show fastmcp

Name: fastmcp
Version: 2.11.3
Summary: The fast, Pythonic way to build MCP servers and clients.
Home-page: https://gofastmcp.com
Author: Jeremiah Lowin
Author-email: 
License: 
Location: C:\Users\amathew\OneDrive - CLEVR\Learning\Zoomcamp\LLMzoomcamp\LLMzoomcamp2025\llmzoomcamp2025\Lib\site-packages
Requires: authlib, cyclopts, exceptiongroup, httpx, mcp, openapi-core, openapi-pydantic, pydantic, pyperclip, python-dotenv, rich
Required-by: 


Version of FastMCP: 2.11.3
Summary: The fast, Pythonic way to build MCP servers and clients.

## Q4. Simple MCP Server
A simple MCP server from the documentation looks like that:

In [14]:
# weather_server.py
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

# @mcp.tool
# def add(a: int, b: int) -> int:
#     """Add two numbers"""
#     return a + b

# if __name__ == "__main__":
#     mcp.run()

In [17]:
@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


@mcp.tool
def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

[08/12/25 23:31:44] WARNING  Tool already exists: get_weather                                   tool_manager.py:169

                    WARNING  Tool already exists: set_weather                                   tool_manager.py:169

In [ ]:
# In-memory "database"
# weather_server.py
import random
from fastmcp import FastMCP

known_weather_data = {"berlin": 20.0}
mcp = FastMCP("Demo 🚀")

In [20]:
# Message to get: Starting MCP server 'Demo 🚀' with transport '<TODO>'

# if __name__ == "__main__":
#     mcp.run()

## Q5. Protocol
There are different ways to communicate with an MCP server. Ours is currently running using standart input/output, which means that the client write something to stdin and read the answer using stdout.

Our weather server is currently running.

This is how we start communitcating with it:

First, we send an initialization request -- this way, we register our client with the server:


Output while running code:



╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Demo 🚀                                           │
│    📦 Transport:       STDIO                                               │  
│                                                                            │
│    📚 Docs:            https://gofastmcp.com                               │
│    🚀 Deploy:          https://fastmcp.cloud                               │
│                                                                            │
│    🏎️  FastMCP version: 2.11.3                                             │
│    🤝 MCP version:     1.12.4                                              │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯



{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
{"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.12.4"}}}
{"jsonrpc": "2.0", "method": "notifications/initialized"}
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
{"jsonrpc":"2.0","id":2,"result":{"tools":[{"name":"get_weather","description":"Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.","inputSchema":{"properties":{"city":{"title":"City","type":"string"}},"required":["city"],"type":"object"},"outputSchema":{"properties":{"result":{"title":"Result","type":"number"}},"required":["result"],"title":"_WrappedResult","type":"object","x-fastmcp-wrap-result":true},"_meta":{"_fastmcp":{"tags":[]}}},{"name":"set_weather","description":"Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.","inputSchema":{"properties":{"city":{"title":"City","type":"string"},"temp":{"title":"Temp","type":"number"}},"required":["city","temp"],"type":"object"},"_meta":{"_fastmcp":{"tags":[]}}}]}}
{"jsonrpc":"2.0","id":3,"method":"tools/call", "params":{"name":"get_weather","arguments":{"city":"Berlin"}}}
{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}

## Q6. Client

We typically don't interact with the server by copy-pasting commands in the terminal.

In practice, we use an MCP Client. Let's implement it.

FastMCP also supports MCP clients:

from fastmcp import Client
async def main():
    async with Client(<TODO>) as mcp_client:

Use the client to get the list of available tools of our script. How does the result look like?

If you're running this code in Jupyter, you need to pass an instance of MCP server to the Client:

import weather_server
async def main():
    async with Client(weather_server.mcp) as mcp_client:


If you run it in a script, you will need to use asyncio:

import asyncio
async def main():
    async with Client("weather_server.py") as mcp_client:
        # ...
if __name__ == "__main__":
    test = asyncio.run(main())


Copy the output with the available tools when filling in the homework form.

In [ ]:
# # async_client.py
# import asyncio
# import weather_mcp_server
# from fastmcp import Client

# async def main():
#     async with Client(weather_mcp_server.mcp) as mcp_client:
#         tools = await mcp_client.list_tools()
#         print(tools)

# if __name__ == "__main__":
#     asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop